# Trial1: 그냥 버트 써서 classifier 통과시키기.

In [ ]:
import json
import pandas as pd
# with open("data/fromgithub2.json","r") as f:
#     df = pd.read_json(f)
#     # posts = json.loads(f.read())

# df = df[["title","time","url"]]
# df["time"] = df["time"].astype("datetime64[s]",)
# df["visited"] = 0
# # df.to_csv("data/fromgithub2.csv")

import json
import pandas as pd

FILE = "data/database1.csv"
df = pd.read_csv(FILE)
df = df[["title","visited"]]


df

In [ ]:
from datetime import datetime

# # features_df = df[["title","time","kids"]]
# # titles = df.loc[df["title"].isna()==False,"title"]
# df.dropna(subset=["title"])
# titles = df["title"]

# # features_df.loc[:,"kids"] = features_df["kids"].apply(lambda x: len(x))
# # features_df["time"] = features_df["time"].astype('datetime64[s]')
# titles.head()

In [ ]:
# # fill dummy data
# y = [1 if i%2==0 else 0 for i in range(len(features))]
# y = np.array(y)

from sklearn.model_selection import train_test_split
RANDOM_STATE = 42

X_train, X_test, y_train, y_test = train_test_split(
    df["title"], df["visited"], test_size=0.33, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel 
from transformers import BertTokenizer, BertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertModel.from_pretrained("bert-base-uncased")


# tokenized = df["title"].apply(
#     (lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_train = X_train.apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_test = X_test.apply(
    (lambda x: tokenizer.encode(x, add_special_tokens=True)))

tmp = max([len(row) for row in tokenized_train])
max_len = max([len(row) for row in tokenized_test])
max_len = max(tmp, max_len)


padded_train = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])
padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])
attention_mask_train = np.where(padded_train!=0 , 1, 0)
attention_mask_test = np.where(padded_test!=0 , 1, 0)

In [ ]:
import torch

output_train = model(
    torch.tensor(padded_train),
    attention_mask=torch.tensor(attention_mask_train))
# output_test = model(torch.tensor(padded_test), attention_mask=attention_mask_test)

X_train = output_train[0][:,0,:].detach().numpy()
X_train.shape

In [ ]:
output_test = model(
    torch.tensor(padded_test),
    attention_mask=torch.tensor(attention_mask_test))

X_test = output_test[0][:,0,:].detach().numpy()
X_test.shape

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

print("DUMMY")
clf = DummyClassifier(random_state=RANDOM_STATE)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)
# cross_val_score(clf, X_test, y_test)

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
# from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.metrics import roc_auc_score

print("LINEAR, LOGISTIC, RIDGE")
# clf = LinearRegression()
# clf = RidgeClassifier()
clfs = [
    LinearRegression(),
    # LogisticRegression(max_iter=2, n_jobs=-1,random_state=RANDOM_STATE),
    # LogisticRegression(),
    LogisticRegression(max_iter=5, solver="sag", n_jobs=-1,random_state=RANDOM_STATE),
    RidgeClassifier(),
    GaussianNB(),
]


for clf in clfs:
    print()
    clf.fit(X_train, y_train)
    print("acc: ", clf.score(X_test, y_test))
    print("crossval: ", cross_val_score(clf, X_test, y_test))
    # y_score = clf.pre(X_test)[:,1]
    y_score = clf.predict(X_test)
    print("auroc: ", roc_auc_score(y_test,y_score))
    # fpr, tpr = roc_curve(y_test, y_score)
    # display(RocCurveDisplay(fpr, tpr, ))


# Trial2: fine tuing bert. with tensorflow

In [ ]:
import pandas as pd
# from sklearn.model_selection import train_test_split

FILE = "data/database1.csv"
RANDOM_STATE = 42

df = pd.read_csv(FILE)
# df[df["url"]==""]

# X_train, X_test, y_train, y_test = train_test_split(
#     df["title"], df["visited"], test_size=0.33, random_state=RANDOM_STATE)

# X_train.shape, X_test.shape, y_train.shape, y_test.shape
# df

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

BERT_TYPE = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)

# tokenized = df["title"].apply(
#     (lambda x: tokenizer(x, return_tensors="np", padding=True)))
# max_len = max([len(row) for row in tokenized])

tokenized = tokenizer(df["title"].to_list(), padding=True, add_special_tokens=True)
tokenized = dict(tokenized)

labels = np.array(df["visited"])  # Label is already an array of 0 and 1
print(len(tokenized["input_ids"]), len(tokenized["input_ids"][0]), labels.shape)

In [ ]:
import tensorflow
from tensorflow.keras.optimizers import Adam
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(BERT_TYPE)
model.compile(optimizer=Adam(3e-5))
model.fit(tokenized, labels)

# 33333... with Pytorch trainer

In [ ]:
import pandas as pd

FILE = "data/database1.csv"
RANDOM_STATE = 42
BERT_TYPE = "distilbert-base-uncased"

df = pd.read_csv(FILE)[["title", "visited"]]
df

In [ ]:
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)

tokenized = tokenizer(df["title"].to_list(), padding=True, add_special_tokens=True)
tokenized = dict(tokenized)
labels = np.array(df["visited"])  # Label is already an array of 0 and 1
print(len(tokenized["input_ids"]), len(tokenized["input_ids"][0]), labels.shape)

In [ ]:
full_df = pd.DataFrame(tokenized)
full_df["visited"] = labels
full_dataset = full_df.to_dict('series')
print(full_dataset)
# print(tokenized["input_ids"])

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(BERT_TYPE, num_labels=2)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_df,
    eval_dataset=eval_df,
    compute_metrics=compute_metrics,
)

trainer.train()

# more fine tuning stuff with tf
- https://team-ak.tistory.com/2
- https://www.kaggle.com/code/manojgadde/beginner-bert-fine-tuning-disaster-tweets

In [ ]:
import pandas as pd

FILE = "data/database1.csv"
RANDOM_STATE = 42
BERT_TYPE = "distilbert-base-uncased"

df = pd.read_csv(FILE)[["title", "visited"]]
# df = df[:200]

import numpy as np
from transformers import DistilBertTokenizer, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)
# tokenized = tokenizer(df["title"].to_list(), return_tensors="np", padding=True)
# print(tokenized)

tokenizer = DistilBertTokenizer.from_pretrained(BERT_TYPE)
tokenized = tokenizer(df["title"].to_list(), 
                padding=True, add_special_tokens=True)

input_ids = np.asarray(tokenized["input_ids"])
attention_mask = np.asarray(tokenized["attention_mask"])
labels = np.asarray(df["visited"])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, mask_train, mask_test \
    = train_test_split(input_ids, labels, attention_mask,
                        test_size=0.2, random_state=RANDOM_STATE)

for data in [X_train, X_test, y_train, y_test, mask_train, mask_test]:
    print(data.shape)

In [ ]:
from transformers import (
    TFBertForSequenceClassification, 
    TFDistilBertForSequenceClassification,
)

bert_original = TFDistilBertForSequenceClassification.from_pretrained(BERT_TYPE)
bert_finetuned = TFDistilBertForSequenceClassification.from_pretrained(BERT_TYPE)

In [ ]:
# X_train.shape, y_train.shape, mask_train.shape
import collections
print(collections.Counter(y_train))

In [ ]:
import tensorflow as tf

# optimizer = tf.keras.optimizers.Adam(learning_rate=2e-3)
# optimizer = tf.keras.optimizers.Adam()
# optimizer = tf.keras.optimizers.Adagrad()
# metric_auc = tf.keras.metrics.AUC(from_logits=False, multi_label=True, num_labels=2)

bert_finetuned.compile(
    # optimizer=optimizer,
    optimizer = tf.keras.optimizers.Adam(3e-5),
    # loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
)
# history = bert_finetuned.fit([X_train, mask_train], y_train, epochs=1)
history = bert_finetuned.fit(
    X_train, y_train, 
    # batch_size=32,
    epochs=3,
)

In [ ]:
# loss, acc = bert_finetuned.evaluate([X_test, mask_test], y_test)
# print(loss, acc)

xxx = bert_finetuned.evaluate(X_test, y_test)
print(xxx)


In [ ]:
from sklearn.metrics import roc_auc_score
preds = bert_finetuned.predict([X_test,mask_test])
y_pred = np.argmax(preds["logits"],axis=1)
print(collections.Counter(y_pred))
roc_auc_score(y_test, y_pred)

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred))

# train model with keras
- https://huggingface.co/docs/transformers/training#train-a-tensorflow-model-with-keras

In [1]:
import pandas as pd

FILE = "data/database1.csv"
RANDOM_STATE = 42
BERT_TYPE = "distilbert-base-uncased"

df = pd.read_csv(FILE)[["title", "visited"]]
# df = df[:200]

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.33, random_state=RANDOM_STATE)
print(train.shape, test.shape)
print(train["visited"].value_counts())

# ############# imbalanced data. smote. ##########
# print("IMBALANCED: smote")
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(random_state=RANDOM_STATE)
# X, y = smote.fit_resample(X, y)
# print(f"{y.count(1)} {y.count(0)}")
# # ############# imbalanced data. dumb method. ##########
# print("IMBALANCED: just make copies")
# tmp = train[train["visited"]==1]
# train = pd.concat([train,tmp])
# train = pd.concat([train,tmp])
# train = pd.concat([train,tmp])
# train = pd.concat([train,tmp])
# display(train["visited"].value_counts())

X_train, y_train = train["title"], train["visited"]
X_test, y_test = test["title"], test["visited"]

display(train["visited"].value_counts())

(999, 2) (493, 2)
visited
0    828
1    171
Name: count, dtype: int64


visited
0    828
1    171
Name: count, dtype: int64

In [2]:
import numpy as np
from transformers import DistilBertTokenizer, AutoTokenizer

def tokenize(X):
    tokenizer = AutoTokenizer.from_pretrained(BERT_TYPE)
    tokenized = tokenizer(X.to_list(), return_tensors="np", padding=True)
    # tokenized = dict(tokenized)
    return dict(tokenized)

def preprocess_all(a,b,c,d):
    """ returns x_train, x_test, y_train, y_test """
    return tokenize(a), tokenize(b), np.array(c), np.array(d)

X_train, X_test, y_train, y_test = preprocess_all(
    X_train, X_test, y_train, y_test
)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained(BERT_TYPE)

# Lower learning rates are often better for fine-tuning transformers
model.compile(
    optimizer=Adam(3e-5),
    metrics=["acc"]
)

model.fit(X_train, y_train, epochs=5)

2023-06-06 19:11:19.086665: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 19:11:19.158934: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 19:11:19.159850: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 19:11:20.223362: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification

Epoch 1/5
32/32 [==============================] - 104s 3s/step - loss: 0.4949 - acc: 0.8258
Epoch 2/5
32/32 [==============================] - 85s 3s/step - loss: 0.4279 - acc: 0.8288
Epoch 3/5
32/32 [==============================] - 87s 3s/step - loss: 0.3559 - acc: 0.8418
Epoch 4/5
32/32 [==============================] - 85s 3s/step - loss: 0.1912 - acc: 0.9369
Epoch 5/5
32/32 [==============================] - 85s 3s/step - loss: 0.0968 - acc: 0.9660


In [4]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred["logits"],axis=1)

tup = model.evaluate(X_test, y_test)
print(tup)

16/16 [==============================] - 13s 683ms/step - loss: 0.7500 - acc: 0.7748
[0.7499707937240601, 0.7748478651046753]


In [6]:
import collections
from sklearn.metrics import roc_auc_score, f1_score
# print(collections.Counter(y_pred))
print(roc_auc_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.5333333333333334
0.20143884892086328
